# Skin Cancer Classification

En el siguiente notebook se expone el desarrollo de una CNN para clasificar fotografías de lunares en función de si son cancerígenas o no. El dataset está compuesto por fotografías en formato ".jpg" separadas en carpetas según si pertenecen al set de entrenamiento o de test y según si son benignas o malignas. El dataset parece complicado puesto que las fotografías han sido tomadas desde distintos ángulos, con distinta luz y algunas imágenes se ven peor debido a que el paciente tenia vello en la zona del lunar.

### Imports

In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import os
import glob
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
np.random.seed(5)

Using TensorFlow backend.


## ETL process

### Load data

In [2]:
benign_train_folder = 'data/train/benign'
malignant_train_folder = 'data/train/malignant'

benign_test_folder = 'data/test/benign'
malignant_test_folder = 'data/test/malignant'

Función para leer todas las imágenes de una carpeta

In [3]:
def read(folder_path):
    data_path = os.path.join(folder_path,'*jpg')
    folder = glob.glob(data_path)
    matrix = []
    for f in folder:
        img = np.asarray(Image.open(f).convert("RGB"))
        matrix.append(img)
    matrix = np.asarray(matrix)
    return matrix

#Create data
X_benign_train = read(benign_train_folder)
X_malignant_train = read(malignant_train_folder)
X_benign_test = read(benign_test_folder)
X_malignant_test = read(malignant_test_folder)

Creamos las etiquetas para cada set de datos; 0 para los benignos y 1 para los malignos.

In [4]:
Y_benign_train = np.zeros(X_benign_train.shape[0])
Y_malignant_train = np.ones(X_malignant_train.shape[0])
Y_benign_test = np.zeros(X_benign_test.shape[0])
Y_malignant_test = np.ones(X_malignant_test.shape[0])

Concatenamos los set de datos y los barajamos, la red funcionaría peor si los datos de entrenamiento estuvieran ordenados en función la salida.

In [5]:
X_train = np.concatenate((X_benign_train, X_malignant_train), axis = 0)
Y_train = np.concatenate((Y_benign_train, Y_malignant_train), axis = 0)
s = np.arange(X_train.shape[0])
np.random.shuffle(s)
X_train = X_train[s]
Y_train = Y_train[s]

X_test = np.concatenate((X_benign_test, X_malignant_test), axis = 0)
Y_test = np.concatenate((Y_benign_test, Y_malignant_test), axis = 0)
s = np.arange(X_test.shape[0])
np.random.shuffle(s)
X_test = X_test[s]
Y_test = Y_test[s]

Normalizaión y transformación de las salidas

In [6]:
#Turn labels into one hot encoding (ya veremos si esto hace falta o si lo dejo)
Y_train = to_categorical(Y_train, num_classes= 2)
Y_test = to_categorical(Y_test, num_classes= 2)

# Normalization
X_train = X_train/255.
X_test = X_test/255.
print(X_train.shape)

(2637, 224, 224, 3)


## Model  

In [7]:
from tensorflow import keras 
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, AveragePooling2D

Se implementa un modelo con 3 capas convolucionales y 3 capas de Maxpooling que vuelcan los resultados en una capa fully connected conectada con una capa de 30 neuronas.

In [8]:
#Model
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, BatchNormalization, Dropout
model = Sequential()
model.add(Conv2D(3, kernel_size=7 ,padding='same', strides=2, activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
#model.add(BatchNormalization())
#model.add(Dropout(0.25))

model.add(Conv2D(3, kernel_size=3,padding='same', strides=2, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
#model.add(BatchNormalization())
#model.add(Dropout(0.25))


from keras import regularizers  
model.add(Flatten())
#model.add(Dropout(0.25))
model.add(Dense(450, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
# model.add(Dropout(0.25))
model.add(Dense(200, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
#model.add(Dropout(0.25))
model.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
#model.add(Dropout(0.25))
model.add(Dense(30, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(2, activation='softmax')) 

KeyboardInterrupt: 

Compilamos con optimizer=adam y loss=categorical_crossentropy. Despues entrenamos el modelo.

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=10, epochs=500)


Por último, evaluamos dicho modelo con los datos de test.

In [ ]:
test_loss, test_acc = model.evaluate(X_test, Y_test, batch_size=10)
print('Test Acc: ', test_acc)
print(':)')

## Otro modelo

In [ ]:
from keras.applications.resnet50 import ResNet50
model = ResNet50(include_top=True,
                 weights= None,
                 input_tensor=None,
                 input_shape=[224,224,3],
                 pooling='avg',
                 classes=2)

model.compile(#optimizer = Adam(lr) ,
              optimizer='adam',
              loss = "binary_crossentropy", 
              metrics=["accuracy"])

model.fit(X_train, Y_train, #validation_split=0.2,
                    epochs= 5, batch_size= 16, #verbose=2, 
                    #callbacks=[learning_rate_reduction]
                   )

In [ ]:
test_loss, test_acc = model.evaluate(X_test, Y_test, batch_size=16)
print('Test Acc: ', test_acc)

## Otro modelo

In [26]:
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, BatchNormalization, Dropout
model = Sequential()
model.add(Conv2D(128, kernel_size=7, activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())

model.add(Conv2D(128, kernel_size=5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
    
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(30, activation='relu'))
model.add(Dense(2, activation='softmax'))        

In [23]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=64, epochs=3)

Epoch 1/3
2637/2637 [==============================] - 811s 308ms/step - loss: 1.3937 - accuracy: 0.6716
Epoch 2/3
2637/2637 [==============================] - 858s 325ms/step - loss: 0.7556 - accuracy: 0.7110
Epoch 3/3
2637/2637 [==============================] - 1250s 474ms/step - loss: 0.8008 - accuracy: 0.7725


In [25]:
test_loss, test_acc = model.evaluate(X_test, Y_test, batch_size=16)
print('Test Acc: ', test_acc)

660/660 [==============================] - 72s 109ms/step
Test Acc:  0.5515151619911194


## Otro modelo

In [11]:
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, BatchNormalization, Dropout
model = Sequential()
model.add(Conv2D(128, kernel_size=7, activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
#model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
#model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
#model.add(Dropout(0.25))
    
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(125, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(30, activation='relu'))
model.add(Dense(2, activation='softmax'))      

In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=16, epochs=30)

Epoch 1/30
2637/2637 [==============================] - 115s 44ms/step - loss: 1.8634 - accuracy: 0.6640
Epoch 2/30
2637/2637 [==============================] - 115s 44ms/step - loss: 0.6778 - accuracy: 0.7114
Epoch 3/30
2637/2637 [==============================] - 115s 44ms/step - loss: 0.5162 - accuracy: 0.7509
Epoch 4/30
2637/2637 [==============================] - 115s 44ms/step - loss: 0.4895 - accuracy: 0.7740
Epoch 5/30
2637/2637 [==============================] - 115s 44ms/step - loss: 0.4443 - accuracy: 0.7865
Epoch 6/30
2637/2637 [==============================] - 115s 44ms/step - loss: 0.4303 - accuracy: 0.7838
Epoch 7/30
2637/2637 [==============================] - 115s 44ms/step - loss: 0.4135 - accuracy: 0.7899
Epoch 8/30
2637/2637 [==============================] - 116s 44ms/step - loss: 0.4023 - accuracy: 0.7998
Epoch 9/30
2637/2637 [==============================] - 3077s 1s/step - loss: 0.3819 - accuracy: 0.8062
Epoch 10/30
2637/2637 [==============================] -

In [13]:
test_loss, test_acc = model.evaluate(X_test, Y_test, batch_size=16)
print('Test Acc: ', test_acc)

660/660 [==============================] - 9s 14ms/step
Test Acc:  0.831818163394928


In [14]:
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, BatchNormalization, Dropout
model = Sequential()
model.add(Conv2D(128, kernel_size=7, activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
#model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
#model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
#model.add(Dropout(0.25))
    
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(125, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(30, activation='relu'))
model.add(Dense(2, activation='softmax')) 

In [15]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=16, epochs=40)

Epoch 1/40
2637/2637 [==============================] - 113s 43ms/step - loss: 2.5502 - accuracy: 0.6799
Epoch 2/40
2637/2637 [==============================] - 112s 43ms/step - loss: 1.0952 - accuracy: 0.7019
Epoch 3/40
2637/2637 [==============================] - 114s 43ms/step - loss: 0.6500 - accuracy: 0.7254
Epoch 4/40
2637/2637 [==============================] - 114s 43ms/step - loss: 0.4625 - accuracy: 0.7774
Epoch 5/40
2637/2637 [==============================] - 114s 43ms/step - loss: 0.4488 - accuracy: 0.7842
Epoch 6/40
2637/2637 [==============================] - 114s 43ms/step - loss: 0.3900 - accuracy: 0.8062
Epoch 7/40
2637/2637 [==============================] - 114s 43ms/step - loss: 0.3923 - accuracy: 0.7986
Epoch 8/40
2637/2637 [==============================] - 114s 43ms/step - loss: 0.3743 - accuracy: 0.8074
Epoch 9/40
2637/2637 [==============================] - 114s 43ms/step - loss: 0.3692 - accuracy: 0.8142
Epoch 10/40
2637/2637 [==============================] 

In [16]:
test_loss, test_acc = model.evaluate(X_test, Y_test, batch_size=16)
print('Test Acc: ', test_acc)

660/660 [==============================] - 9s 14ms/step
Test Acc:  0.7878788113594055


## El mismo pero 5 epochs

In [13]:
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, BatchNormalization, Dropout
model = Sequential()
model.add(Conv2D(128, kernel_size=7, activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
#model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
#model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
#model.add(Dropout(0.25))
    
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(125, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(30, activation='relu'))
model.add(Dense(2, activation='softmax'))   

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=16, epochs=5)

Epoch 1/5
2637/2637 [==============================] - 735s 279ms/step - loss: 3.1937 - accuracy: 0.6447
Epoch 2/5
2637/2637 [==============================] - 734s 278ms/step - loss: 0.7880 - accuracy: 0.7270
Epoch 3/5
2637/2637 [==============================] - 734s 278ms/step - loss: 0.5560 - accuracy: 0.7497
Epoch 4/5
2637/2637 [==============================] - 735s 279ms/step - loss: 0.4681 - accuracy: 0.7812
Epoch 5/5
2637/2637 [==============================] - 731s 277ms/step - loss: 0.4642 - accuracy: 0.7740


In [15]:
test_loss, test_acc = model.evaluate(X_test, Y_test, batch_size=16)
print('Test Acc: ', test_acc)

660/660 [==============================] - 55s 84ms/step
Test Acc:  0.7863636612892151


## Por dios que sea el bueno

In [37]:
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, BatchNormalization, Dropout
model = Sequential()
model.add(Conv2D(128, kernel_size=7, activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
#model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
#model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
#model.add(Dropout(0.25))
    
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(125, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(30, activation='relu'))
model.add(Dense(2, activation='softmax')) 

In [38]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=16, epochs=8)

Epoch 1/8
2637/2637 [==============================] - 116s 44ms/step - loss: 2.0556 - accuracy: 0.6712
Epoch 2/8
2637/2637 [==============================] - 119s 45ms/step - loss: 0.8020 - accuracy: 0.6997
Epoch 3/8
2637/2637 [==============================] - 118s 45ms/step - loss: 0.5314 - accuracy: 0.7539
Epoch 4/8
2637/2637 [==============================] - 116s 44ms/step - loss: 0.4765 - accuracy: 0.7717
Epoch 5/8
2637/2637 [==============================] - 116s 44ms/step - loss: 0.4432 - accuracy: 0.7785
Epoch 6/8
2637/2637 [==============================] - 115s 44ms/step - loss: 0.4277 - accuracy: 0.7945
Epoch 7/8
2637/2637 [==============================] - 178s 68ms/step - loss: 0.4096 - accuracy: 0.8013
Epoch 8/8
2637/2637 [==============================] - 326s 124ms/step - loss: 0.3988 - accuracy: 0.8032


In [39]:
test_loss, test_acc = model.evaluate(X_test, Y_test, batch_size=16)
print('Test Acc: ', test_acc)

660/660 [==============================] - 11s 16ms/step
Test Acc:  0.8439394235610962


## Más capas

In [15]:
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, BatchNormalization, Dropout
model = Sequential()
model.add(Conv2D(128, kernel_size=7, activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
#model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
#model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
#model.add(Dropout(0.25))
    
model.add(Flatten())
#model.add(Dropout(0.25))
model.add(Dense(450, activation='relu'))
#model.add(Dropout(0.25))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(125, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(30, activation='relu'))
model.add(Dense(2, activation='softmax')) 

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=16, epochs=5)

Epoch 1/5
2637/2637 [==============================] - 785s 298ms/step - loss: 1.9183 - accuracy: 0.6469
Epoch 2/5
2637/2637 [==============================] - 776s 294ms/step - loss: 0.5752 - accuracy: 0.7345
Epoch 3/5
2637/2637 [==============================] - 769s 292ms/step - loss: 0.4700 - accuracy: 0.7660
Epoch 4/5
2637/2637 [==============================] - 774s 294ms/step - loss: 0.4556 - accuracy: 0.7763
Epoch 5/5
2637/2637 [==============================] - 768s 291ms/step - loss: 0.4141 - accuracy: 0.8043


In [17]:
test_loss, test_acc = model.evaluate(X_test, Y_test, batch_size=16)
print('Test Acc: ', test_acc)

660/660 [==============================] - 56s 85ms/step
Test Acc:  0.8257575631141663


## Más capas, 10 epochs

In [18]:
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, BatchNormalization, Dropout
model = Sequential()
model.add(Conv2D(128, kernel_size=7, activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
#model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
#model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
#model.add(Dropout(0.25))
    
model.add(Flatten())
#model.add(Dropout(0.25))
model.add(Dense(450, activation='relu'))
#model.add(Dropout(0.25))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(125, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(30, activation='relu'))
model.add(Dense(2, activation='softmax')) 

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=16, epochs=10)

Epoch 1/10
2637/2637 [==============================] - 755s 286ms/step - loss: 2.3214 - accuracy: 0.6606
Epoch 2/10
2637/2637 [==============================] - 750s 284ms/step - loss: 0.7202 - accuracy: 0.7190
Epoch 3/10
2637/2637 [==============================] - 750s 284ms/step - loss: 0.5970 - accuracy: 0.7262
Epoch 4/10
2637/2637 [==============================] - 749s 284ms/step - loss: 0.5151 - accuracy: 0.7501
Epoch 5/10
2637/2637 [==============================] - 748s 284ms/step - loss: 0.4743 - accuracy: 0.7679
Epoch 6/10
2637/2637 [==============================] - 751s 285ms/step - loss: 0.4640 - accuracy: 0.7801
Epoch 7/10
2637/2637 [==============================] - 747s 283ms/step - loss: 0.3988 - accuracy: 0.8157
Epoch 8/10
2637/2637 [==============================] - 748s 284ms/step - loss: 0.3863 - accuracy: 0.8074
Epoch 9/10
2637/2637 [==============================] - 750s 284ms/step - loss: 0.3738 - accuracy: 0.8233
Epoch 10/10
2637/2637 [=======================

In [20]:
test_loss, test_acc = model.evaluate(X_test, Y_test, batch_size=16)
print('Test Acc: ', test_acc)

660/660 [==============================] - 56s 85ms/step
Test Acc:  0.8075757622718811


In [21]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=16, epochs=20)

Epoch 1/20
2637/2637 [==============================] - 771s 292ms/step - loss: 0.5042 - accuracy: 0.7649
Epoch 2/20
2637/2637 [==============================] - 752s 285ms/step - loss: 0.6695 - accuracy: 0.7235
Epoch 3/20
2637/2637 [==============================] - 752s 285ms/step - loss: 0.4353 - accuracy: 0.7861
Epoch 4/20
2637/2637 [==============================] - 760s 288ms/step - loss: 0.4507 - accuracy: 0.7907
Epoch 5/20
2637/2637 [==============================] - 750s 284ms/step - loss: 0.4159 - accuracy: 0.8062
Epoch 6/20
2637/2637 [==============================] - 751s 285ms/step - loss: 0.4279 - accuracy: 0.7964
Epoch 7/20
2637/2637 [==============================] - 750s 285ms/step - loss: 0.3629 - accuracy: 0.8165
Epoch 8/20
2637/2637 [==============================] - 750s 284ms/step - loss: 0.3677 - accuracy: 0.8294
Epoch 9/20
2637/2637 [==============================] - 751s 285ms/step - loss: 0.3574 - accuracy: 0.8278
Epoch 10/20
2637/2637 [=======================

In [22]:
test_loss, test_acc = model.evaluate(X_test, Y_test, batch_size=16)
print('Test Acc: ', test_acc)

660/660 [==============================] - 57s 87ms/step
Test Acc:  0.7378787994384766


In [23]:
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, BatchNormalization, Dropout
model = Sequential()
model.add(Conv2D(128, kernel_size=7, activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
#model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
#model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
#model.add(Dropout(0.25))
    
model.add(Flatten())
#model.add(Dropout(0.25))
model.add(Dense(450, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(125, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(30, activation='relu'))
model.add(Dense(2, activation='softmax')) 

In [24]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=16, epochs=20)

Epoch 1/20
2637/2637 [==============================] - 766s 290ms/step - loss: 1.6544 - accuracy: 0.6401
Epoch 2/20
2637/2637 [==============================] - 755s 286ms/step - loss: 0.6532 - accuracy: 0.7167
Epoch 3/20
2637/2637 [==============================] - 756s 287ms/step - loss: 0.5255 - accuracy: 0.7429
Epoch 4/20
2637/2637 [==============================] - 753s 285ms/step - loss: 0.5014 - accuracy: 0.7732
Epoch 5/20
2637/2637 [==============================] - 753s 286ms/step - loss: 0.4745 - accuracy: 0.7588
Epoch 6/20
2637/2637 [==============================] - 757s 287ms/step - loss: 0.4290 - accuracy: 0.7819
Epoch 7/20
2637/2637 [==============================] - 756s 287ms/step - loss: 0.4118 - accuracy: 0.7914
Epoch 8/20
2637/2637 [==============================] - 756s 287ms/step - loss: 0.3929 - accuracy: 0.8039
Epoch 9/20
2637/2637 [==============================] - 758s 288ms/step - loss: 0.3848 - accuracy: 0.8104
Epoch 10/20
2637/2637 [=======================

In [25]:
test_loss, test_acc = model.evaluate(X_test, Y_test, batch_size=16)
print('Test Acc: ', test_acc)

660/660 [==============================] - 56s 85ms/step
Test Acc:  0.789393961429596


## Enésimo intento

In [8]:
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, BatchNormalization, Dropout
model = Sequential()
model.add(Conv2D(128, kernel_size=7, activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
#model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
#model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
#model.add(Dropout(0.25))
    
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(450, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(125, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(30, activation='relu'))
model.add(Dense(2, activation='softmax')) 

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=16, epochs=20)

Epoch 1/20
2637/2637 [==============================] - 127s 48ms/step - loss: 2.0363 - accuracy: 0.6644
Epoch 2/20
2637/2637 [==============================] - 123s 46ms/step - loss: 0.7226 - accuracy: 0.7080
Epoch 3/20
2637/2637 [==============================] - 124s 47ms/step - loss: 0.5746 - accuracy: 0.7345
Epoch 4/20
2637/2637 [==============================] - 122s 46ms/step - loss: 0.5075 - accuracy: 0.7539
Epoch 5/20
2637/2637 [==============================] - 125s 47ms/step - loss: 0.4624 - accuracy: 0.7763
Epoch 6/20
2637/2637 [==============================] - 127s 48ms/step - loss: 0.4183 - accuracy: 0.7922
Epoch 7/20
2637/2637 [==============================] - 136s 52ms/step - loss: 0.3936 - accuracy: 0.8039
Epoch 8/20
2637/2637 [==============================] - 229s 87ms/step - loss: 0.4140 - accuracy: 0.7960
Epoch 9/20
2637/2637 [==============================] - 217s 82ms/step - loss: 0.3852 - accuracy: 0.8096
Epoch 10/20
2637/2637 [==============================] 

In [10]:
test_loss, test_acc = model.evaluate(X_test, Y_test, batch_size=16)
print('Test Acc: ', test_acc)

660/660 [==============================] - 12s 19ms/step
Test Acc:  0.6984848380088806


In [11]:
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, BatchNormalization, Dropout
model = Sequential()
model.add(Conv2D(128, kernel_size=7, activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
model.add(Dropout(0.25))
    
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(450, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(125, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(30, activation='relu'))
model.add(Dense(2, activation='softmax')) 

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=16, epochs=20)

Epoch 1/20
2637/2637 [==============================] - 133s 50ms/step - loss: 1.4008 - accuracy: 0.6621
Epoch 2/20
2637/2637 [==============================] - 132s 50ms/step - loss: 0.5898 - accuracy: 0.7122
Epoch 3/20
2637/2637 [==============================] - 133s 50ms/step - loss: 0.5453 - accuracy: 0.7588
Epoch 4/20
2637/2637 [==============================] - 133s 50ms/step - loss: 0.4834 - accuracy: 0.7634
Epoch 5/20
2637/2637 [==============================] - 133s 50ms/step - loss: 0.4370 - accuracy: 0.7835
Epoch 6/20
2637/2637 [==============================] - 132s 50ms/step - loss: 0.4760 - accuracy: 0.7755
Epoch 7/20
2637/2637 [==============================] - 132s 50ms/step - loss: 0.4514 - accuracy: 0.7789
Epoch 8/20
2637/2637 [==============================] - 132s 50ms/step - loss: 0.4097 - accuracy: 0.7880
Epoch 9/20
2637/2637 [==============================] - 132s 50ms/step - loss: 0.4004 - accuracy: 0.8005
Epoch 10/20
2637/2637 [==============================] 

In [14]:
test_loss, test_acc = model.evaluate(X_test, Y_test, batch_size=16)
print('Test Acc: ', test_acc)

660/660 [==============================] - 9s 14ms/step
Test Acc:  0.821212112903595


In [8]:
#el mismo, con 40 epochs
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, BatchNormalization, Dropout
model = Sequential()
model.add(Conv2D(128, kernel_size=7, activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
model.add(Dropout(0.25))
    
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(450, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(125, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(30, activation='relu'))
model.add(Dense(2, activation='softmax')) 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=16, epochs=40)
test_loss, test_acc = model.evaluate(X_test, Y_test, batch_size=16)
print('Test Acc: ', test_acc)

Epoch 1/40
2637/2637 [==============================] - 155s 59ms/step - loss: 1.6733 - accuracy: 0.6519
Epoch 2/40
2637/2637 [==============================] - 210s 80ms/step - loss: 0.6318 - accuracy: 0.7232
Epoch 3/40
2637/2637 [==============================] - 131s 50ms/step - loss: 0.5358 - accuracy: 0.7452
Epoch 4/40
2637/2637 [==============================] - 132s 50ms/step - loss: 0.4953 - accuracy: 0.7527
Epoch 5/40
2637/2637 [==============================] - 133s 51ms/step - loss: 0.4397 - accuracy: 0.7770
Epoch 6/40
2637/2637 [==============================] - 131s 50ms/step - loss: 0.4353 - accuracy: 0.7804
Epoch 7/40
2637/2637 [==============================] - 131s 50ms/step - loss: 0.4086 - accuracy: 0.7986
Epoch 8/40
2637/2637 [==============================] - 131s 50ms/step - loss: 0.3895 - accuracy: 0.8043
Epoch 9/40
2637/2637 [==============================] - 131s 50ms/step - loss: 0.3909 - accuracy: 0.8187
Epoch 10/40
2637/2637 [==============================] 

## Ahora con regularización

In [10]:
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, BatchNormalization, Dropout
from keras import regularizers
model = Sequential()
model.add(Conv2D(128, kernel_size=7, activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
model.add(Dropout(0.25))
    
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(450, activation='relu', kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.25))
model.add(Dense(250, activation='relu', kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.25))
model.add(Dense(125, activation='relu', kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.25))
model.add(Dense(30, activation='relu', kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)))
model.add(Dense(2, activation='softmax')) 

In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=16, epochs=40)

Epoch 1/40
2637/2637 [==============================] - 135s 51ms/step - loss: 13.8785 - accuracy: 0.6469
Epoch 2/40
2637/2637 [==============================] - 135s 51ms/step - loss: 5.1810 - accuracy: 0.7603
Epoch 3/40
2637/2637 [==============================] - 136s 51ms/step - loss: 3.3128 - accuracy: 0.7835
Epoch 4/40
2637/2637 [==============================] - 135s 51ms/step - loss: 2.5566 - accuracy: 0.7789
Epoch 5/40
2637/2637 [==============================] - 135s 51ms/step - loss: 2.0030 - accuracy: 0.7785
Epoch 6/40
2637/2637 [==============================] - 135s 51ms/step - loss: 1.6255 - accuracy: 0.7766
Epoch 7/40
2637/2637 [==============================] - 135s 51ms/step - loss: 1.3115 - accuracy: 0.7819
Epoch 8/40
2637/2637 [==============================] - 135s 51ms/step - loss: 1.1171 - accuracy: 0.7876
Epoch 9/40
2637/2637 [==============================] - 135s 51ms/step - loss: 0.9770 - accuracy: 0.7903
Epoch 10/40
2637/2637 [==============================]

In [12]:
test_loss, test_acc = model.evaluate(X_test, Y_test, batch_size=16)
print('Test Acc: ', test_acc)

660/660 [==============================] - 21s 33ms/step
Test Acc:  0.789393961429596


## Nuevo enfoque

In [22]:
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, BatchNormalization, Dropout
from keras import regularizers
model = Sequential()
model.add(Conv2D(128, kernel_size=7, activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(BatchNormalization())
model.add(Dropout(0.25))
    
model.add(Flatten())
model.add(Dropout(0.50))
model.add(Dense(450, activation='relu', kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.50))
model.add(Dense(450, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(125, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(30, activation='relu'))
model.add(Dense(2, activation='softmax')) 

In [23]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=16, epochs=30)

Epoch 1/30
2637/2637 [==============================] - 161s 61ms/step - loss: 10.1517 - accuracy: 0.6416
Epoch 2/30
2637/2637 [==============================] - 161s 61ms/step - loss: 2.8305 - accuracy: 0.7296
Epoch 3/30
2637/2637 [==============================] - 143s 54ms/step - loss: 1.1710 - accuracy: 0.7641
Epoch 4/30
2637/2637 [==============================] - 143s 54ms/step - loss: 1.1571 - accuracy: 0.7622
Epoch 5/30
2637/2637 [==============================] - 234s 89ms/step - loss: 1.0566 - accuracy: 0.7721
Epoch 6/30
2637/2637 [==============================] - 255s 97ms/step - loss: 1.1460 - accuracy: 0.7740
Epoch 7/30
2637/2637 [==============================] - 252s 96ms/step - loss: 0.9602 - accuracy: 0.7945
Epoch 8/30
2637/2637 [==============================] - 262s 99ms/step - loss: 1.1534 - accuracy: 0.7759
Epoch 9/30
2637/2637 [==============================] - 251s 95ms/step - loss: 1.1099 - accuracy: 0.7770
Epoch 10/30
2637/2637 [==============================]

In [24]:
test_loss, test_acc = model.evaluate(X_test[0:330], Y_test[0:330], batch_size=16)
print('Test Acc: ', test_acc)

330/330 [==============================] - 7s 22ms/step
Test Acc:  0.760606050491333
